<a href="https://colab.research.google.com/github/jennifer-z-1/DXC-Tech-1A/blob/main/DXC1A_Destiny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install PyPDF2 langchain transformers faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 891.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [23]:
import PyPDF2
from google.colab import drive
drive.mount('/content/drive')


def extract_text_from_pdf(pdf_file_path):
    with open(pdf_file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

# My file path
pdf_file_path = '/content/drive/My Drive/EU_AI_Act.pdf'
document_text = extract_text_from_pdf(pdf_file_path)
print(document_text)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
BRIEFING  
EU Legislation in Progress  
 
EPRS | European Parliamentary Research  Service  
Author: Tambiama  Madiega  
Members' Research Service 
PE 698.792  –  March 2024  EN 
Artificial intelligence act  
OVERVIEW  
European Union lawmakers reached a political agreement on the draft artificial intelligence (AI) act 
in December 2023. Proposed by the European Commission in April  2021, t he draft AI act, the first 
binding  worldwide  horizontal regulation  on AI, sets a common framework for the use and supply of 
AI systems in the EU. It  offers  a classification for AI sy stems with different requirements and 
obligations tailored on a 'risk-based approach '. Some AI systems presenting 'unacceptable ' risks are 
prohibited. A wide range of 'high -risk' AI systems that can have a detrimental impact on people' s 
health, safety or o n their fundamental righ

In [13]:
def chunk_text(text, max_length=1000):
    sentences = text.split('.')
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) < max_length:
            current_chunk += sentence + "."
        else:
            chunks.append(current_chunk)
            current_chunk = sentence + "."

    if current_chunk:  # Add the last chunk
        chunks.append(current_chunk)

    return chunks

chunks = chunk_text(document_text)


In [14]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load a pre-trained sentence transformer model
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

def get_embeddings(texts):
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = model_output.last_hidden_state.mean(dim=1)
    return embeddings

embeddings = get_embeddings(chunks)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [15]:
import faiss
import numpy as np

# Convert PyTorch tensor to numpy array
embeddings_np = embeddings.cpu().numpy()

# Initialize FAISS index
dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
index.add(embeddings_np)

# For furture
faiss.write_index(index, 'faiss_index.idx')


In [16]:
def search(query, index, tokenizer, model, chunks, top_k=5):
    query_embedding = get_embeddings([query]).cpu().numpy()
    _, I = index.search(query_embedding, top_k)  # Get indices of top_k results
    return [chunks[i] for i in I[0]]

# Testing query
query = "regulations on AI systems"
top_chunks = search(query, index, tokenizer, model, chunks)
print(top_chunks)


[' toys, aviation, cars, medical devices, lifts).  \n\uf0d8 Systems deployed in eight specific areas  specified in Annex (e.g. law enforcement),  \nwhic h the Commission could update as necessary through delegated acts.   \nSuch high- risk AI systems would have to comply with a range of requirements  particularly on risk \nmanagement, testing, technical robustness, data training and data governance, transparency,  \nhuman oversight, and cybersecurity  before being placed on the market or put  into service. AI \nsystems that conform to new harmonised EU standards  would benefit from a presumption of \nconformity with the draft AI act requirements. \nThird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots), \nemotion recognition systems, biometric categorisation systems, and AI systems that generate or \nmanipulate image, audio or video content (i.e. deepfakes ), would be subject to a limi ted set of \ntransparency obligations.', '  \nThe Co

In [17]:
pip install cohere


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.2 MB/s eta 0:00:00


In [18]:
import cohere

# Set up Cohere client
cohere_api_key = 'B4PNT1RIE9JkYKt1T7ZSJD6zK5bXzvyirlvIOsNa'
co = cohere.Client(cohere_api_key)


In [19]:
def get_embeddings(text_chunks):
    response = co.embed(texts=text_chunks, model='large')
    return response.embeddings

# Get embeddings for the chunks
embeddings = get_embeddings(chunks)


In [20]:
pip install faiss-cpu


In [21]:
import faiss
import numpy as np

# Convert embeddings to numpy array
embedding_matrix = np.array(embeddings)

# Initialize FAISS index
dimension = embedding_matrix.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
index.add(embedding_matrix)


In [22]:
def search(query, index, chunks, top_k=5):
    query_embedding = co.embed(texts=[query], model='large').embeddings[0]
    query_embedding = np.array([query_embedding])

    # Search in the FAISS index
    distances, indices = index.search(query_embedding, top_k)

    # Retrieve the most relevant chunks
    relevant_chunks = [chunks[i] for i in indices[0]]
    return relevant_chunks

# using a query
query = "regulations on AI systems"
top_chunks = search(query, index, chunks)
print(top_chunks)


[' toys, aviation, cars, medical devices, lifts).  \n\uf0d8 Systems deployed in eight specific areas  specified in Annex (e.g. law enforcement),  \nwhic h the Commission could update as necessary through delegated acts.   \nSuch high- risk AI systems would have to comply with a range of requirements  particularly on risk \nmanagement, testing, technical robustness, data training and data governance, transparency,  \nhuman oversight, and cybersecurity  before being placed on the market or put  into service. AI \nsystems that conform to new harmonised EU standards  would benefit from a presumption of \nconformity with the draft AI act requirements. \nThird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots), \nemotion recognition systems, biometric categorisation systems, and AI systems that generate or \nmanipulate image, audio or video content (i.e. deepfakes ), would be subject to a limi ted set of \ntransparency obligations.', '  \nThe Co